# Dusty wave

This notebook demonstrates setting up the dusty wave test.

The dust and gas are co-located in a box with uniform density. There is an initial uniform differential velocity between the dust and gas.

## Initialization

Import required modules.

In [ ]:
%load_ext blackcellmagic

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import phantomsetup
import plonk

In [ ]:
%matplotlib widget

Set constants.

In [ ]:
igas = phantomsetup.defaults.PARTICLE_TYPE['igas']
idust = phantomsetup.defaults.PARTICLE_TYPE['idust']

## Parameters

We set the parameters for the setup.

### Prefix

We set the file name prefix, such that the dump file is `prefix_00000.tmp.h5` and the in file is `prefix.in`.

In [ ]:
prefix = 'dustywave'

### Equation of state

An `ieos` of 1 sets the globally isothermal equation of state. The sound speed is the only free parameter.

In [ ]:
ieos = 1
sound_speed = 1.0

### Boundary

The boundary of the box as (xmin, xmax, ymin, ymax, zmin, zmax).

In [ ]:
box_boundary = (-0.5, 0.5, -0.5, 0.5, -0.5, 0.5)

### Resolution

The number of gas particles.

In [ ]:
number_of_particles_gas = 50_000

The number of dust particles in each species.

In [ ]:
number_of_particles_dust = 10_000

### Density

The initial uniform density of the gas.

In [ ]:
density_gas = 1.0

The dust-to-gas ratio for each dust species.

In [ ]:
dust_to_gas_ratio = (0.01, 0.02, 0.03, 0.04, 0.05)

### Dust

The dust drag method. Options are "Epstein/Stokes", "K_const", or "ts_const".

In [ ]:
drag_method = 'K_const'

The constant drag coefficient. This is not required for Epstein/Stokes drag.

In [ ]:
K_code = 1.0

The grain size of each dust species. We don't set this for constant drag as it is not required.

In [ ]:
grain_size = ()

The intrinsic grain density.

In [ ]:
grain_density = 3.0

### Velocity

The initial delta in uniform velocity between gas and dust.

In [ ]:
velocity_delta = 1.0

## Instantiate phantomsetup object

We instantiate the `phantom.Setup` object.

In [ ]:
setup = phantomsetup.Setup()

Then we add the previously defined parameters to this object.

### File prefix

In [ ]:
setup.prefix = prefix

### Set units

We convert unit strings to cgs values to pass to the `set_units` method.

In [ ]:
length_unit = phantomsetup.units.unit_string_to_cgs('au')
mass_unit = phantomsetup.units.unit_string_to_cgs('solarm')
time_unit = phantomsetup.units.unit_string_to_cgs('year')

In [ ]:
setup.set_units(length=length_unit, mass=mass_unit, time=time_unit)

### Set equation of state

In [ ]:
setup.set_equation_of_state(ieos=ieos, polyk=sound_speed ** 2)

### Set dust

Here we call the `set_dust` method differently depending on the drag type. First we get the number of species from the `dust_to_gas_ratio` tuple.

In [ ]:
number_of_dust_species = len(dust_to_gas_ratio)

And we set the dust density from the dust-to-gas ratio.

In [ ]:
density_dust = [eps * density_gas for eps in dust_to_gas_ratio]

Then we initialize the dust via the `set_dust` method.

In [ ]:
if drag_method == 'Epstein/Stokes':
    setup.set_dust(
        dust_method='largegrains',
        drag_method=drag_method,
        grain_size=grain_size,
        grain_density=grain_density,
    )

elif drag_method in ('K_const', 'ts_const'):
    setup.set_dust(
        dust_method='largegrains',
        drag_method=drag_method,
        drag_constant=K_code,
        number_of_dust_species=number_of_dust_species,
    )

### Set boundary

This sets the boundary, and the boundary conditions to periodic.

In [ ]:
setup.set_boundary(box_boundary, periodic=True)

### Make a box of particles

The `Box` class sets up a box of particles in a uniform spatial distribution with an arbitrary velocity field.

Set lattice.

In [ ]:
lattice = 'cubic'

#### Velocity distribution

We first define a velocity distribution. The velocities are all sine waves.

In [ ]:
box_width = (box_boundary[1] - box_boundary[0])
kwave = 2 * np.pi / box_width
ampl = 1.0e-4


def velocity_distribution(x, y, z):
    """Initialize velocity perturbation."""
    vx, vy, vz = np.zeros(x.shape), np.zeros(y.shape), np.zeros(z.shape)
    vx = ampl * np.sin(kwave * (x + box_width / 2))
    return vx, vy, vz

#### Make a gas box

Then we instantiate a `Box` object, add particles, and add it to the setup.

In [ ]:
boxes = list()
box = phantomsetup.Box(
    box_boundary=box_boundary,
    particle_type=igas,
    number_of_particles=number_of_particles_gas,
    density=density_gas,
    velocity_distribution=velocity_distribution,
    lattice=lattice,
)
boxes.append(box)

#### Make dust boxes

We do the same for each dust species.

Then we iterate over each of the dust species.

In [ ]:
for idx in range(number_of_dust_species):
    box = phantomsetup.Box(
        box_boundary=box_boundary,
        particle_type=idust + idx,
        number_of_particles=number_of_particles_dust,
        density=density_dust[idx],
        velocity_distribution=velocity_distribution,
        lattice=lattice,
    )
    boxes.append(box)

### Add extra quantities to particles

Phantom requires the $\alpha$ viscosity parameter array. We set it to zero. Note that the array is single precision.

In [ ]:
for box in boxes:
    alpha = np.zeros(box.number_of_particles, dtype=np.single)
    box.set_array('alpha', alpha)

### Add boxes to setup

In [ ]:
for box in boxes:
    setup.add_container(box)

## Write to file

Now that we are happy with the setup, write the "temporary" dump file with the initial conditions and the Phantom "in" file.

First we set a working directory for the simulation.

In [ ]:
working_dir = pathlib.Path(f'~/runs/{prefix}').expanduser()

In [ ]:
setup.write_dump_file(directory=working_dir)
setup.write_in_file(directory=working_dir)

## Compile Phantom

You can start a Phantom calculation from these two files but you must compile Phantom with the correct Makefile variables. We can use the `phantom_compile_command` method to show how Phantom would be compiled.

In [ ]:
print(setup.phantom_compile_command())

To compile Phantom we can use the `compile_phantom` method to compile Phantom. For example, the following will compile Phantom located in `phantom_dir` and copy the binary to `working_dir`.

```
result = setup.compile_phantom(
    phantom_dir='~/repos/phantom',
    working_dir=working_dir
)
```

## Check with Plonk

In [ ]:
snap = plonk.load_snap(working_dir / f"{prefix}_00000.tmp.h5")
n_species = len(snap.properties['grain size']) + 1

#### Positions

In [ ]:
fig, axes = plt.subplots(ncols=n_species//2, nrows=2)
for idx in range(n_species):
    subsnap = snap[snap["dust_id"] == idx]
    plonk.visualize.plot(
        x_coordinate=subsnap["x"],
        y_coordinate=subsnap["y"],
        extent=box_boundary,
        particle_mass=subsnap["m"],
        smoothing_length=subsnap["h"],
        axis=axes.ravel()[idx],
    )

#### Velocities

In [ ]:
factor = 1.5
extent = (box_boundary[0], box_boundary[1], -ampl * factor, ampl * factor)

fig, axes = plt.subplots(ncols=n_species//2, nrows=2)
for idx in range(n_species):
    axis = axes.ravel()[idx]
    subsnap = snap[snap["dust_id"] == idx]
    plonk.visualize.plot(
        x_coordinate=subsnap["x"],
        y_coordinate=subsnap["vx"],
        extent=extent,
        particle_mass=subsnap["m"],
        smoothing_length=subsnap["h"],
        axis=axis,
    )
    axis.set_aspect('auto')